# Guardar y reanudar el aprendizaje en Tensorflow

Trabajaremos con el DataSet Pima Indian Diabetes, que proporciona datos sobre el estado de salud de las personas. Además de las indicaciones fisiológicas, hay información en los datos sobre la presencia de diabetes en una persona. Esto será considerado una clase.

In [26]:
import pandas as pd
import wget

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

wget.download(url, "pima-indians-diabetes.data.csv")

dataset = pd.read_csv("pima-indians-diabetes.data.csv", header=None)
dataset.columns = [
    "number_times_pregnant", "plasma_glucose_concent", "blood_pressure",
    "skin_thickness", "insulin", "BMI", "diabetes_pedigree", "age", "has_diabetes"
]

print(dataset.head())


100% [..............................................................................] 23278 / 23278   number_times_pregnant  plasma_glucose_concent  blood_pressure  \
0                      6                     148              72   
1                      1                      85              66   
2                      8                     183              64   
3                      1                      89              66   
4                      0                     137              40   

   skin_thickness  insulin   BMI  diabetes_pedigree  age  has_diabetes  
0              35        0  33.6              0.627   50             1  
1              29        0  26.6              0.351   31             0  
2               0        0  23.3              0.672   32             1  
3              23       94  28.1              0.167   21             0  
4              35      168  43.1              2.288   33             1  


In [27]:
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(10)
X = dataset.drop(['has_diabetes'], axis=1).values
y = dataset['has_diabetes'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}, Labels ratio: {y_train.mean():.2f}")

Train shape: (614, 8), Test shape: (154, 8), Labels ratio: 0.34


## Guardar y cargar modelos

In [28]:
import tensorflow as tf

In [29]:
def get_compiled_model():
    """
    La función devuelve el modelo compilado para la clasificación binaria
    """
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(24, input_dim=8, activation='relu'))
    model.add(tf.keras.layers.Dense(24, activation='relu'))
    model.add(tf.keras.layers.Dense(24, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = get_compiled_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=64, verbose=1)

Epoch 1/150
10/10 [==============================] - 1s 31ms/step - loss: 1.1363 - accuracy: 0.5831 - val_loss: 0.8429 - val_accuracy: 0.6104
Epoch 2/150
10/10 [==============================] - 0s 5ms/step - loss: 0.6953 - accuracy: 0.6189 - val_loss: 0.7025 - val_accuracy: 0.5455
Epoch 3/150
10/10 [==============================] - 0s 5ms/step - loss: 0.6684 - accuracy: 0.6270 - val_loss: 0.7003 - val_accuracy: 0.5455
Epoch 4/150
10/10 [==============================] - 0s 6ms/step - loss: 0.6438 - accuracy: 0.6710 - val_loss: 0.6450 - val_accuracy: 0.6364
Epoch 5/150
10/10 [==============================] - 0s 5ms/step - loss: 0.6283 - accuracy: 0.6678 - val_loss: 0.6194 - val_accuracy: 0.6688
Epoch 6/150
10/10 [==============================] - 0s 5ms/step - loss: 0.5913 - accuracy: 0.6971 - val_loss: 0.6406 - val_accuracy: 0.6299
Epoch 7/150
10/10 [==============================] - 0s 5ms/step - loss: 0.6033 - accuracy: 0.6743 - val_loss: 0.6208 - val_accuracy: 0.6494
Epoch 8/150


### Guardar el modelo

**Tf es responsable de registrar los puntos de control en keras.keras.callbacks.Model Check point**. Como se puede ver en la ruta del módulo -- se refiere a los colegas (callbacks).

Tiene un parámetro obligatorio : el nombre del Checkpoint. Se puede formatear utilizando variables de Servicio (ejemplo a continuación). De parámetros opcionales pero útiles:

* monitor (str): nombre de la métrica que desea "seguir"
* verbose -- imprimir información adicional
* save_best_only (bool): guardar el Checkpoint solo después de mejorar la métrica especificada en monitor
* save_weight_only (bool): además de los pesos del modelo, también puede guardar su arquitectura y el estado del optimizador. En los ejemplos lo haremos, a menos que se diga lo contrario. El valor predeterminado es false.
* mode (str): si la métrica necesita maximizar -- max, de lo contrario -- min.

Utilizamos **hdf5** -- el formato principal para keras. Para Tensorflow -- es **ckpt**.

In [30]:
from pathlib import Path
path = Path("model_1")
path.mkdir(exist_ok=True) # crear una carpeta en el disco
cpt_filename = "{epoch:02d}_checkpoint_{val_loss:.2f}.hdf5"  
cpt_path = str(path.joinpath(cpt_filename))


checkpoint = tf.keras.callbacks.ModelCheckpoint(cpt_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [31]:
for epoch in range(4):
    print(cpt_path.format(epoch=epoch, val_loss=np.random.rand()))

model_1\00_checkpoint_0.38.hdf5
model_1\01_checkpoint_0.67.hdf5
model_1\02_checkpoint_0.39.hdf5
model_1\03_checkpoint_0.22.hdf5


In [32]:
model = get_compiled_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=64, verbose=0, 
          callbacks=[checkpoint])


Epoch 1: val_loss improved from inf to 1.33874, saving model to model_1\01_checkpoint_1.34.hdf5

Epoch 2: val_loss improved from 1.33874 to 0.75880, saving model to model_1\02_checkpoint_0.76.hdf5

Epoch 3: val_loss did not improve from 0.75880


C:\Users\Ульяна\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 4: val_loss improved from 0.75880 to 0.69966, saving model to model_1\04_checkpoint_0.70.hdf5

Epoch 5: val_loss did not improve from 0.69966

Epoch 6: val_loss did not improve from 0.69966

Epoch 7: val_loss did not improve from 0.69966

Epoch 8: val_loss did not improve from 0.69966

Epoch 9: val_loss improved from 0.69966 to 0.67091, saving model to model_1\09_checkpoint_0.67.hdf5

Epoch 10: val_loss did not improve from 0.67091

Epoch 11: val_loss did not improve from 0.67091

Epoch 12: val_loss improved from 0.67091 to 0.65106, saving model to model_1\12_checkpoint_0.65.hdf5

Epoch 13: val_loss did not improve from 0.65106

Epoch 14: val_loss did not improve from 0.65106

Epoch 15: val_loss did not improve from 0.65106

Epoch 16: val_loss did not improve from 0.65106

Epoch 17: val_loss did not improve from 0.65106

Epoch 18: val_loss did not improve from 0.65106

Epoch 19: val_loss improved from 0.65106 to 0.63680, saving model to model_1\19_checkpoint_0.64.hdf5

Epoch 20:

### Carga del modelo

Hay dos formas de cargar el modelo. Definir un modelo y cargar pesos en él, utilizando *model.load_weights()*. O tf.keras.models.load_model para crear automáticamente un modelo y cargar pesos (podemos hacerlo porque al crear Checkpoint, configuramos save_weights_only=False, es decir, guardamos la descripción del modelo).

In [34]:
model = get_compiled_model() # definimos un modelo inicializado aleatoriamente
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of random model {acc*100 :.2f}%")

5/5 [==============================] - 0s 2ms/step - loss: 13.5945 - accuracy: 0.6234
Accuracy of random model 62.34%


In [37]:
model.load_weights("model_1\\108_checkpoint_0.58.hdf5") # cargando pesos
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of restored model {acc*100 :.2f}%")

5/5 [==============================] - 0s 1ms/step - loss: 0.5821 - accuracy: 0.7468
Accuracy of restored model 74.68%


In [38]:
model = tf.keras.models.load_model("model_1\\108_checkpoint_0.58.hdf5")
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of restored model {acc*100 :.2f}%")

5/5 [==============================] - 0s 2ms/step - loss: 0.5821 - accuracy: 0.7468
Accuracy of restored model 74.68%


### Guardar el modelo en un Checkpoint

Si no formatea el nombre del archivo, todos los puntos de control se escribirán en él:

In [39]:
path = Path("model_2")
path.mkdir(exist_ok=True)
cpt_filename = "checkpoint_best.hdf5"
cpt_path =str(path / cpt_filename)

checkpoint = tf.keras.callbacks.ModelCheckpoint(cpt_path, monitor='val_loss', verbose=1, save_best_only=True, mode='max')

model = get_compiled_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=64, verbose=0, 
          callbacks=[checkpoint])


Epoch 1: val_loss improved from -inf to 0.69071, saving model to model_2\checkpoint_best.hdf5

Epoch 2: val_loss did not improve from 0.69071

Epoch 3: val_loss did not improve from 0.69071

Epoch 4: val_loss did not improve from 0.69071

Epoch 5: val_loss improved from 0.69071 to 0.74862, saving model to model_2\checkpoint_best.hdf5

Epoch 6: val_loss did not improve from 0.74862

Epoch 7: val_loss did not improve from 0.74862

Epoch 8: val_loss did not improve from 0.74862

Epoch 9: val_loss did not improve from 0.74862

Epoch 10: val_loss did not improve from 0.74862

Epoch 11: val_loss did not improve from 0.74862

Epoch 12: val_loss did not improve from 0.74862

Epoch 13: val_loss did not improve from 0.74862

Epoch 14: val_loss did not improve from 0.74862

Epoch 15: val_loss did not improve from 0.74862

Epoch 16: val_loss did not improve from 0.74862

Epoch 17: val_loss did not improve from 0.74862

Epoch 18: val_loss did not improve from 0.74862

Epoch 19: val_loss did not im

In [40]:
ls model_2

 ’®¬ ў гбва®©бвўҐ C ­Ґ Ё¬ҐҐв ¬ҐвЄЁ.
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : 70A1-5EFA

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ c:\Users\“«мп­ \Desktop\jupiter para git\skillbox_neuron_frameworks\model_2

13.02.2024  18:21    <DIR>          .
13.02.2024  18:21    <DIR>          ..
13.02.2024  18:21            58я424 checkpoint_best.hdf5
               1 д ©«®ў         58я424 Ў ©в
               2 Ї Ї®Є  92я380я217я344 Ў ©в бў®Ў®¤­®


In [41]:
model = tf.keras.models.load_model("model_2/checkpoint_best.hdf5")
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of restored model {acc*100 :.2f}%")

5/5 [==============================] - 0s 2ms/step - loss: 0.8189 - accuracy: 0.6364
Accuracy of restored model 63.64%


### Continuar el proceso de aprendizaje

In [42]:
# podemos continuar el proceso de aprendizaje, tenga en cuenta initial_epoch
model = tf.keras.models.load_model("model_2/checkpoint_best.hdf5")
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=300, batch_size=64, verbose=0, 
          callbacks=[checkpoint], initial_epoch=150)


Epoch 151: val_loss did not improve from 0.81888

Epoch 152: val_loss did not improve from 0.81888

Epoch 153: val_loss did not improve from 0.81888

Epoch 154: val_loss did not improve from 0.81888

Epoch 155: val_loss did not improve from 0.81888

Epoch 156: val_loss did not improve from 0.81888

Epoch 157: val_loss did not improve from 0.81888

Epoch 158: val_loss did not improve from 0.81888

Epoch 159: val_loss did not improve from 0.81888

Epoch 160: val_loss did not improve from 0.81888

Epoch 161: val_loss did not improve from 0.81888

Epoch 162: val_loss did not improve from 0.81888

Epoch 163: val_loss did not improve from 0.81888

Epoch 164: val_loss did not improve from 0.81888

Epoch 165: val_loss did not improve from 0.81888

Epoch 166: val_loss did not improve from 0.81888

Epoch 167: val_loss did not improve from 0.81888

Epoch 168: val_loss did not improve from 0.81888

Epoch 169: val_loss did not improve from 0.81888

Epoch 170: val_loss did not improve from 0.81888
